**1. Implementação**

**1.1. Importações e Definições de Parâmetros**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Importando a biblioteca para leitura dos dados
import pandas as pd
import numpy as np

# Importando função para separar o dataset em TREINO e TESTE
from sklearn.model_selection import train_test_split 

# Importando as classes do sklearn para padronização dos dados
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Importando os construtores dos modelos 
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

# Importando as métricas a serem utilizadas
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Importando bibliotecas para visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt


%matplotlib inline


**Importando a base de dados**

Depois de importar a base de dados, fazemos uma breve analise observando se existem valores nulos, quais são os tipos das variáveis (método df.info) e entendendo quais são as escalas de cada uma das variáveis (método df.mean()).

In [ ]:
df = pd.read_csv('../input/mobile-price-classification/train.csv')

In [ ]:
df.head()

In [ ]:
df.info() ## não temos dados faltantes, todos os dados são do tipo numérico

In [ ]:
df.mean()  ## com as medias podemos ver que os números possuem escalas diferentes de valor

In [ ]:
df.describe() ## avaliando os valores do dataset

In [ ]:
df_sem_label = df.drop(['price_range'], axis=1) ## separando o price_range do restante do dataset

df_sem_label

In [ ]:
sc = StandardScaler() ## chamando a função StanderdScaler para fazer a padronização dos dados numéricos

In [ ]:
sc.fit(df_sem_label) ## treinando o modelo a partir dos dados sem o label de price_range

In [ ]:
sc.mean_  ## observando o que foi aprendido a partir do treino

In [ ]:
sc.var_  ## observando o que foi aprendido a partir do treino

In [ ]:
df_norm = sc.transform(df_sem_label) ## normalizando os valores das variveis do dataset a partir do uso do metodo StanderdScaler

In [ ]:
df_norm ## analisando os valores normalizados


**Analisando a melhor quantidade de clusters usando a técnica WSCC**


Falando de maneira matemática, quando usamos o **WSCC (do inglês within-clusters sum-of-squares)** estamos em busca de uma quantidade de agrupamentos no qual a **soma dos quadrados intra-clusters** seja a menor.

Tratando-se do **KMeans** da biblioteca **scikit_learn** o cálculo do WSCC é processado e após isso ele retorna o nome de inertia.

In [ ]:
wcss = []

for i in range(1,10):
    kmeans = KMeans(n_clusters=i, max_iter=300) ## chamando a função do kmeans e atribuindo o numero de clusters com o numero maximo de 300 iterações
    kmeans.fit(df_norm) ## aplicando o K-means na base de dados
    wcss.append(kmeans.inertia_)

**2. Soluções - Parte 01:**


**2.1. K-Means: Cluesterização com Análise não Supervisionada
da pergunta 1-A:**

Temos o melhor número de clusters quando o número de wcss e o de clusters cai drasticamente.

No caso do gráfico abaixo o **nosso número ideal de clusters seria 3.**


**a. Sua análise levou a mais ou menos clusteres que o esperado?**

Na minha analise o número ideal de clusters seria 3, contudo, analisando o df original o numero correto de clusters seria 4, conforme a analise abaixo.

In [ ]:
plt.plot(range(1,10), wcss)
plt.title("Analisando o numero de clusters")
plt.xlabel("Numero de clusters")
plt.ylabel("WCSS")
plt.show()


## o numero de clusters em que temos o melhor wcss e 3

**Comparando com a quantidade ideal de clusters no df original que seriam 4**

In [ ]:
df['price_range'].drop_duplicates() ## quantidade original de modelos foram 4

**Agregando o numero de clusters criados ao DF original**

In [ ]:
clusterizacao = KMeans(n_clusters = 3,  max_iter=300) ## criando o modelo
clusterizacao.fit(df_norm) # fazendo a aplicação do modelo na base de dados

In [ ]:
df['KMeans_Clusters'] = clusterizacao.labels_ ## atribuindo os clusters ao objeto original
df.head()

**da pergunta 1-B:**

**b. Baseado nos valores das amostras pertencentes a cada um dos clusteres formados, o que eles significam?**

Como usamos a **técnica de PCA**, todas as variáveis foram resumidas em dois componentes, sendo que cada um desses dois componentes possuem os valores principais de cada cluster **(1,2,3).**

Nestes dois componentes nós temos os valores que serão usados para plotar cada ponto no cluster ao qual ele pertence.

**Visualizando todos os clusters de maneira grafica**

Para isso vamos usar a **análise de componentes principais (PCA).**

A análise de componentes principais consiste em técnica que transforma um determinado grupo de variáveis originais em outro grupo com a mesma dimensão que chamamos de componentes principais.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2) ## como usaremos um grafico com duas dimensões (x, y) usamos então um modelo que tenha dois componentes principais
pca_df = pca.fit_transform(df_norm) ## para conseguir os dois componentes principais usamos a função fit_transform com o dataset normalizado
pca_df_principal = pd.DataFrame(data = pca_df, columns = ['Primeiro_componente', 'Segundo_componente']) ## com o recurso do Pandas chamado DataFrame, faço o uso para transformar os componentes principais em um dataframe e renomeio as colunas
pca_labels_celulares = pd.concat([pca_df_principal, df[['KMeans_Clusters']]], axis=1) ## depois com a função Concat, concateno as colunas com os clusters criados chamada de "KMeans_Clusters"

pca_labels_celulares

In [ ]:
fig = plt.figure(figsize = (12,12))

ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Primeiro_Componente', fontsize=15)
ax.set_ylabel('Segundo_Componente', fontsize=15)
ax.set_title('Componentes Principais', fontsize=20)

color_theme = np.array(["green", "yellow", "orange"])
ax.scatter(x = pca_labels_celulares.Primeiro_componente, y=pca_labels_celulares.Segundo_componente, 
           c=color_theme[pca_labels_celulares.KMeans_Clusters], s=50)

plt.show()

**3. Soluções - Parte 02:**

**3.1. KNN: Classificação - Análise Supervisionada**

In [ ]:
df

In [ ]:
df.info()

**3.2. Questão 2.a. Quais serão as métricas utilizadas?**

A métrica a ser adotada será a acurácia de classificação.

Temos também já predefinido como Label a variável **Intervelo de Preço (price_range)** e como demais variáveis a compor esta classificação optamos pela escolha de atributos físicos (Hardware) e Tecnológicos que compõem os aparelhos mais modernos, sendo elas: **Capacidade de Bateria (battery_power), bluetooth (blue), Sensibilidade ao toque (touch_screen), Conexão de rede (wifi), Memória Interna (int_memory), Dupla entrada de Chip (dual_sim), Largura de Pixels (px_width), Altura de Pixels (px_height).**


In [ ]:
df1 = df[["price_range", "battery_power", "blue", "touch_screen", "wifi", "int_memory", "dual_sim", "px_width", "px_height"]] #Cria outro DF apenas com as variáveis escolhidas
df1

In [ ]:
# Para fazerem divisão de TREINO, TESTE usando 70/30 e semente aleatória = 42
X = df.drop(columns=["battery_power", "blue", "touch_screen", "wifi", "int_memory", "dual_sim", "px_width", "px_height"])
y = df["price_range"]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

Como a dataframe é composto apenas por valores numéricos não será necessário o uso do OneHotEncoder

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.fit_transform(X_test)

**3.3. Análise da Acurácia da Classificação**

A princípio realizei um teste afim de encontrar o melhor número de vizinhos, através do experimento abaixo, que irá demostrar um gráfico de linha:

In [ ]:
mean_scores = [] #Lista que recebe a acurácia média de obtida para cada K
for k in range(1,42): #Intervalo de variação de K vizinhos
    scores = []
    for i in range(60):
        X_train, X_test, y_train, y_test = train_test_split(X,y)
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train,y_train)
        accuracy = model.score(X_test,y_test)
        scores.append(accuracy)
    mean_scores.append(np.mean(scores))

#Gráfico de linha
plt.plot(np.arange(1,42),mean_scores)
plt.yticks([])
plt.title("Acurácias do k-NN por número de vizinhos")
plt.xlabel("Número de vizinhos")
plt.ylabel("Acurácia")
plt.show()

Neste código estabeleci uma **lista []** onde serão salvas as acurácias. Determinei o número de **k vizinhos de 1 à 42** e determinei que isto seja **repetido 60 vezes,** rodando o treinamento e a validação.

Observei que a partir de **15 vizinhos** o nível de acurácia não apresenta grandes alterações, apesar de sim aumentar. O KNN pode demonstrar modelos diferentes a cada execução e sendo a acurácia uma variável aleatória de acordo com uma probabilidade, decidi **manter o valor de 15 vizinhos.** Porém considerando esta situação para ser estudada futuramente e entender o porque isto ocorre.